In [1]:
import pandas as pd
import numpy as np
import datetime

import GeoFeatureGeneration as gfg

def drop_all_0_row(df):
    # x1 = df[(df==0).all(axis=1)]
    # x2 = 
    return df.drop(index=df[(df==0).all(axis=1)].index)

stay = pd.read_csv('./Data/Output/Stay/153.csv', index_col=0)
stay['stime'] = pd.to_datetime(stay['stime'])
print(stay.columns)

Index(['userID', 'stime', 'grid', 'etime', 'lon', 'lat', 'duration', 'stayid'], dtype='object')


In [2]:
gFeaturePath = './Data/Output/MultipleFeatures/PoIFeature.csv'
PoIFeature = pd.read_csv(gFeaturePath, index_col=0)
PoIFeature['grid'] = PoIFeature.index

stay = stay.merge(PoIFeature, on='grid', how='left').fillna(0)
print(stay.columns)

Index(['userID', 'stime', 'grid', 'etime', 'lon', 'lat', 'duration', 'stayid',
       '0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0',
       '10.0', '11.0', '12.0', '13.0'],
      dtype='object')


In [4]:
stay = stay.apply(gfg.GenerateTimeFeature, col='stime', axis=1)
# stay.head()

stay['stimestamp'] = stay['stime'].astype('int64') // 1e9
# stay.head()

s1 = stay.groupby(pd.Grouper(key='stime', freq='M'))

i = 0
maxrow = 128
print(stay.shape)


(3258, 29)


In [8]:
for g in s1:
    df = g[1].copy()
    print(df.columns)

    df.drop(['stime', 'etime', 'lon', 'lat', 'stayid'], axis=1, inplace=True)
    print(df.columns)

Index(['userID', 'stime', 'grid', 'etime', 'lon', 'lat', 'duration', 'stayid',
       '0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0',
       '10.0', '11.0', '12.0', '13.0', 'weekofyear', 'dayofweek', 'dayofyear',
       'quarter', 'month', 'hour', 'stimestamp'],
      dtype='object')
Index(['userID', 'grid', 'duration', '0.0', '1.0', '2.0', '3.0', '4.0', '5.0',
       '6.0', '7.0', '8.0', '9.0', '10.0', '11.0', '12.0', '13.0',
       'weekofyear', 'dayofweek', 'dayofyear', 'quarter', 'month', 'hour',
       'stimestamp'],
      dtype='object')
Index(['userID', 'stime', 'grid', 'etime', 'lon', 'lat', 'duration', 'stayid',
       '0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0',
       '10.0', '11.0', '12.0', '13.0', 'weekofyear', 'dayofweek', 'dayofyear',
       'quarter', 'month', 'hour', 'stimestamp'],
      dtype='object')
Index(['userID', 'grid', 'duration', '0.0', '1.0', '2.0', '3.0', '4.0', '5.0',
       '6.0', '7.0', '8.0', '9.0', '10.0',

In [6]:
result = np.empty((0, maxrow, stay.shape[1]))
# print(result.shape)

# print(s1.shape)
# s1 = drop_all_0_row(s1)
# print(s1.shape)

for g in s1:
    key = g[0].month
    # print(type(key))
    # .to_list()
    # value = g[1][['grid', 'stimestamp', 'duration']].values
    # print(g[1].columns)
    df = g[1].copy()

    # print("before drop all 0 {}".format(df.shape))
    df  = drop_all_0_row(df)
    # print("after drop all 0 {}".format(df.shape))

    value = df.values
    # print(value.shape)
    
    # value.dropna(axis=0, how='all', inplace=True)
    # 去除全零的行。
    # 由于含有datetime格式的数据，所以无法使用下面的语句来去掉全零行。
    # value = value[value.sum(axis=1)!=0,:]
    # 如果行数为0，也就是说没有轨迹点。那么就跳过。
    if value.shape[0] == 0:
        continue
    # 将轨迹填充为相同的形状。
    if value.shape[0] > maxrow:
        continue
    else:
        value = np.pad(array=value, pad_width=((0,maxrow-value.shape[0]),(0,0)), mode='constant')
    
    # print("value {}".format(value.shape))
    # print(result.shape)
    # d = np.concatenate((a[np.newaxis,:], b[np.newaxis,:]), axis=0)
    result = np.concatenate((result, value[np.newaxis,:]), axis=0)
    # if i < 2:
        
    #     print(value)
    #     i += 1
    # print(type(value))
    # output type is <class 'numpy.ndarray'>
    # np.set_printoptions(suppress=True)
    # print(value)
print(result.shape)


(52, 128, 29)


In [9]:
def np_3d_to_csv(data, 
                 path, 
                 datatype='float'):
    import csv
    a2d = data.reshape(data.shape[0], -1)
    with open(path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(a2d)


In [ ]:
np_3d_to_csv(result, "output.csv")

In [8]:
a2d = result.reshape(result.shape[0], -1)
# np.savetxt('{}.csv'.format(153), a2d, delimiter="," )
import csv
with open("output.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(a2d)
print(result.shape)

(52, 128, 29)


In [21]:
import csv
import numpy as np

# 从csv文件读取2D数组
with open("output.csv", "r") as f:
    reader = csv.reader(f)
    a2d = np.array(list(reader)).astype("float")GetAddressByApply

# 将2D数组转换为3D数组
a = a2d.reshape(-1, 128, 3)
print(a.shape)

(52, 128, 3)


In [ ]:
import pandas as pd

def drop_all_0_row(df):
    # x1 = df[(df==0).all(axis=1)]
    # x2 = 
    return df.drop(index=df[(df==0).all(axis=1)].index)

def tte(user):
    stay = pd.read_csv('./Data/Output/Stay/{}.csv'.format(user), index_col=0)
    stay['stime'] = pd.to_datetime(stay['stime'])
    s1 = stay.groupby(pd.Grouper(key='stime', freq='M'))

    gridList = []
    for g in s1:
        key = g[0].month
        # print(type(key))
        value = g[1]['grid'].to_list()
        gridList.append(value)
        # print(key)
        # print(value)
    gridpd = pd.DataFrame(gridList)
    gridpd = drop_all_0_row(gridpd)
    gridpd.dropna(axis=0, how='all', inplace=True)
    print(gridpd.shape)
    return gridpd

cantor 配对函数

In [6]:
import math
def cantor_pairing_function(x, y):
    """
    计算Cantor对数
    :param x: 第一个数
    :param y: 第二个数
    :return: Cantor对数
    """
    return ((x + y) * (x + y + 1) // 2 + y)

def cantor_unpair(z):
    w = (math.sqrt(8 * z + 1) - 1) // 2
    t = w * (w + 1) // 2
    y = z - t
    x = w - y
    return int(x), int(y)

x = 4
y = 7

print(cantor_pairing_function(x, y))

73


In [2]:
a = 7
b = 4
print(cantor_pairing_function(a, b))

70


In [5]:
for x in range(0, 11):
    for y in range(0, 11):
        print(cantor_pairing_function(x, y))

0
2
5
9
14
20
27
35
44
54
65
1
4
8
13
19
26
34
43
53
64
76
3
7
12
18
25
33
42
52
63
75
88
6
11
17
24
32
41
51
62
74
87
101
10
16
23
31
40
50
61
73
86
100
115
15
22
30
39
49
60
72
85
99
114
130
21
29
38
48
59
71
84
98
113
129
146
28
37
47
58
70
83
97
112
128
145
163
36
46
57
69
82
96
111
127
144
162
181
45
56
68
81
95
110
126
143
161
180
200
55
67
80
94
109
125
142
160
179
199
220


In [3]:
x = 4

print(x % 2)

print(x / 2)

0
2.0


In [14]:
x = -7

if x >= 0:
    x = 2 * x
else:
    x = 2 * abs(x) - 1

print(x)

13


In [15]:
x = 13

if x % 2 == 0:
    x = x / 2
else:
    x = - ((x + 1) / 2)
    
print(int(x))

-7


In [20]:
import math
def CantorPairingFunction(x, y):
    """_summary_
    先对x,y使用折叠函数，然后再计算2个数的cantor配对函数的值。
    Args:
        x (int): 整数。
        y (int): 整数。

    Returns:
        int: 返回cantor配对数。
    """
    if x >= 0:
        x = 2 * x
    else:
        x = 2 * abs(x) - 1
    
    if y >= 0:
        y = 2 * y
    else:
        y = 2 * abs(y) - 1

    return ((x + y) * (x + y + 1) // 2 + y)

def CantorPairingInverseFunction(z):
    """_summary_
    先计算cantor配对函数反函数，然后x,y使用折叠反函数。
    Args:
        z (int): 两个数的cantor配对数值。

    Returns:
        x (int): 整数。
        y (int): 整数。
    """
    if z < 0 :
        print('CantorPairingInverseFunction input z is out of range.')
        return 0, 0
    
    w = (math.sqrt(8 * z + 1) - 1) // 2
    t = w * (w + 1) // 2
    y = z - t
    x = w - y
    
    if x % 2 == 0:
        x = x / 2
    else:
        x = -((x + 1) / 2)
    
    if y % 2 == 0:
        y = y / 2
    else:
        y = -((y + 1) / 2)

    return int(x), int(y)

In [27]:
CantorPairingFunction(-2,-1)

11

In [28]:
x,y = CantorPairingInverseFunction(11)
print(x,y)

-2 -1


In [3]:
import pandas as pd
import polars as pl
import numpy as np
import time

df1 = pl.read_csv('./Data/Output/PoiFeature.csv', has_header=True)
df1.head(3)

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,longitude,latitude,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,…,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2541.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,1.0,115.482481,39.65181,0.004289,0.016499,0.003141,-0.00076,-0.009566,-0.008357,-0.091427,0.010202,-0.038074,-0.018119,-0.014617,-0.046119,-0.09011,0.039288,-0.017326,-0.018314,-0.024455,-0.000881,0.012357,-0.02212,…,-0.006907,0.035788,-0.056014,-0.002544,0.047614,-0.005141,-0.001159,0.040743,0.042059,0.006464,0.037266,0.03376,0.01275,-0.014765,0.077446,0.024541,0.045374,-0.072902,-0.102396,-0.043924,0.017011,0.033909,0.07826,0.006621,0.003874,0.05384,-0.016581,0.010054,0.023912,-0.010222,-0.037931,-0.010941,0.003338,-0.01872,0.009969,0.006657,0.043991
2684.0,0.0,0.0,1.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,115.494264,39.65181,0.001293,0.025325,-0.029833,-0.030759,-0.029629,-0.004699,0.008305,-0.001472,-0.04342,-0.024059,-0.044851,0.012433,-0.033121,0.012506,-0.026177,0.009624,-0.02996,-0.011991,-0.016942,0.016075,…,-0.020418,0.028713,0.003886,0.004295,0.024014,0.013459,-0.035204,0.013323,0.035572,0.021832,0.02056,-0.003609,0.015416,0.016945,0.03776,-0.040684,0.046689,-0.06929,-0.037012,-0.025192,0.012876,-0.006196,0.043972,0.02394,-0.004437,0.04758,-0.054088,0.018877,0.05218,-0.004291,-0.028057,0.021559,0.009324,0.019704,0.020393,0.004436,0.020841
2686.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,115.482481,39.660803,0.004289,0.016499,0.003141,-0.00076,-0.009566,-0.008357,-0.091427,0.010202,-0.038074,-0.018119,-0.014617,-0.046119,-0.09011,0.039288,-0.017326,-0.018314,-0.024455,-0.000881,0.012357,-0.02212,…,-0.006907,0.035788,-0.056014,-0.002544,0.047614,-0.005141,-0.001159,0.040743,0.042059,0.006464,0.037266,0.03376,0.01275,-0.014765,0.077446,0.024541,0.045374,-0.072902,-0.102396,-0.043924,0.017011,0.033909,0.07826,0.006621,0.003874,0.05384,-0.016581,0.010054,0.023912,-0.010222,-0.037931,-0.010941,0.003338,-0.01872,0.009969,0.006657,0.043991


In [4]:
df1 = df1.rename({df1.columns[0]: "grid"})
df1.head(3)

grid,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,longitude,latitude,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,…,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2541.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,1.0,115.482481,39.65181,0.004289,0.016499,0.003141,-0.00076,-0.009566,-0.008357,-0.091427,0.010202,-0.038074,-0.018119,-0.014617,-0.046119,-0.09011,0.039288,-0.017326,-0.018314,-0.024455,-0.000881,0.012357,-0.02212,…,-0.006907,0.035788,-0.056014,-0.002544,0.047614,-0.005141,-0.001159,0.040743,0.042059,0.006464,0.037266,0.03376,0.01275,-0.014765,0.077446,0.024541,0.045374,-0.072902,-0.102396,-0.043924,0.017011,0.033909,0.07826,0.006621,0.003874,0.05384,-0.016581,0.010054,0.023912,-0.010222,-0.037931,-0.010941,0.003338,-0.01872,0.009969,0.006657,0.043991
2684.0,0.0,0.0,1.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,115.494264,39.65181,0.001293,0.025325,-0.029833,-0.030759,-0.029629,-0.004699,0.008305,-0.001472,-0.04342,-0.024059,-0.044851,0.012433,-0.033121,0.012506,-0.026177,0.009624,-0.02996,-0.011991,-0.016942,0.016075,…,-0.020418,0.028713,0.003886,0.004295,0.024014,0.013459,-0.035204,0.013323,0.035572,0.021832,0.02056,-0.003609,0.015416,0.016945,0.03776,-0.040684,0.046689,-0.06929,-0.037012,-0.025192,0.012876,-0.006196,0.043972,0.02394,-0.004437,0.04758,-0.054088,0.018877,0.05218,-0.004291,-0.028057,0.021559,0.009324,0.019704,0.020393,0.004436,0.020841
2686.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,115.482481,39.660803,0.004289,0.016499,0.003141,-0.00076,-0.009566,-0.008357,-0.091427,0.010202,-0.038074,-0.018119,-0.014617,-0.046119,-0.09011,0.039288,-0.017326,-0.018314,-0.024455,-0.000881,0.012357,-0.02212,…,-0.006907,0.035788,-0.056014,-0.002544,0.047614,-0.005141,-0.001159,0.040743,0.042059,0.006464,0.037266,0.03376,0.01275,-0.014765,0.077446,0.024541,0.045374,-0.072902,-0.102396,-0.043924,0.017011,0.033909,0.07826,0.006621,0.003874,0.05384,-0.016581,0.010054,0.023912,-0.010222,-0.037931,-0.010941,0.003338,-0.01872,0.009969,0.006657,0.043991


In [6]:
df1 = df1.with_columns(df1["grid"].cast(pl.Int32))
df1.head(3)

grid,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,longitude,latitude,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,…,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2541,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,1.0,115.482481,39.65181,0.004289,0.016499,0.003141,-0.00076,-0.009566,-0.008357,-0.091427,0.010202,-0.038074,-0.018119,-0.014617,-0.046119,-0.09011,0.039288,-0.017326,-0.018314,-0.024455,-0.000881,0.012357,-0.02212,…,-0.006907,0.035788,-0.056014,-0.002544,0.047614,-0.005141,-0.001159,0.040743,0.042059,0.006464,0.037266,0.03376,0.01275,-0.014765,0.077446,0.024541,0.045374,-0.072902,-0.102396,-0.043924,0.017011,0.033909,0.07826,0.006621,0.003874,0.05384,-0.016581,0.010054,0.023912,-0.010222,-0.037931,-0.010941,0.003338,-0.01872,0.009969,0.006657,0.043991
2684,0.0,0.0,1.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,115.494264,39.65181,0.001293,0.025325,-0.029833,-0.030759,-0.029629,-0.004699,0.008305,-0.001472,-0.04342,-0.024059,-0.044851,0.012433,-0.033121,0.012506,-0.026177,0.009624,-0.02996,-0.011991,-0.016942,0.016075,…,-0.020418,0.028713,0.003886,0.004295,0.024014,0.013459,-0.035204,0.013323,0.035572,0.021832,0.02056,-0.003609,0.015416,0.016945,0.03776,-0.040684,0.046689,-0.06929,-0.037012,-0.025192,0.012876,-0.006196,0.043972,0.02394,-0.004437,0.04758,-0.054088,0.018877,0.05218,-0.004291,-0.028057,0.021559,0.009324,0.019704,0.020393,0.004436,0.020841
2686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,115.482481,39.660803,0.004289,0.016499,0.003141,-0.00076,-0.009566,-0.008357,-0.091427,0.010202,-0.038074,-0.018119,-0.014617,-0.046119,-0.09011,0.039288,-0.017326,-0.018314,-0.024455,-0.000881,0.012357,-0.02212,…,-0.006907,0.035788,-0.056014,-0.002544,0.047614,-0.005141,-0.001159,0.040743,0.042059,0.006464,0.037266,0.03376,0.01275,-0.014765,0.077446,0.024541,0.045374,-0.072902,-0.102396,-0.043924,0.017011,0.033909,0.07826,0.006621,0.003874,0.05384,-0.016581,0.010054,0.023912,-0.010222,-0.037931,-0.010941,0.003338,-0.01872,0.009969,0.006657,0.043991


In [6]:
import pandas as pd

userTrajectory = pd.read_csv("./Data/Output/TrajectoryFeature/000.csv", 
                                 index_col=0,
                                 parse_dates=['entireTime'], nrows=10)

print(userTrajectory.columns)

Index(['latitude', 'longitude', 'alt', 'entireTime', 'loncol', 'latcol',
       'grid', 'userID', '0.0', '1.0',
       ...
       '502', '503', '504', '505', '506', '507', '508', '509', '510', '511'],
      dtype='object', length=536)


In [11]:
import polars as pl

userTrajectory = pl.read_csv("./Data/Output/TrajectoryFeature/000.csv", n_rows=10)

# print(userTrajectory.columns)
userTrajectory.head(3)

,latitude,longitude,alt,entireTime,loncol,latcol,grid,userID,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,longitude_right,latitude_right,0,1,2,3,4,5,6,7,8,9,10,11,…,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
i64,f64,f64,i64,str,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,40.007453,116.319888,155,"""2009-04-20 01:30:39""",78,68,42914,0,0.0,31.0,0.0,8.0,1.0,6.0,9.0,0.0,7.0,4.0,6.0,1.0,7.0,24.0,116.319078,40.011538,0.06699,0.006994,0.013861,-0.01067,-0.036054,-0.003406,-0.04291,0.028169,0.02093,-0.021014,-0.044312,-0.037629,…,-0.002603,0.064733,-0.029632,0.005119,0.019319,0.003828,-0.040464,0.040389,0.05743,-0.005433,-0.006662,0.012598,0.022417,-0.004811,0.044321,0.009152,0.003701,-0.076212,-0.051264,-0.04497,-0.002828,-0.026574,0.049401,0.013856,-0.003176,0.029433,-0.035817,0.008347,-0.022139,0.01779,-0.068493,0.014312,-0.003235,-0.026788,-0.029159,-0.021948,0.009429
1,40.007462,116.319895,155,"""2009-04-20 01:30:44""",78,68,42914,0,0.0,31.0,0.0,8.0,1.0,6.0,9.0,0.0,7.0,4.0,6.0,1.0,7.0,24.0,116.319078,40.011538,0.06699,0.006994,0.013861,-0.01067,-0.036054,-0.003406,-0.04291,0.028169,0.02093,-0.021014,-0.044312,-0.037629,…,-0.002603,0.064733,-0.029632,0.005119,0.019319,0.003828,-0.040464,0.040389,0.05743,-0.005433,-0.006662,0.012598,0.022417,-0.004811,0.044321,0.009152,0.003701,-0.076212,-0.051264,-0.04497,-0.002828,-0.026574,0.049401,0.013856,-0.003176,0.029433,-0.035817,0.008347,-0.022139,0.01779,-0.068493,0.014312,-0.003235,-0.026788,-0.029159,-0.021948,0.009429
2,40.007452,116.319878,156,"""2009-04-20 01:30:49""",78,68,42914,0,0.0,31.0,0.0,8.0,1.0,6.0,9.0,0.0,7.0,4.0,6.0,1.0,7.0,24.0,116.319078,40.011538,0.06699,0.006994,0.013861,-0.01067,-0.036054,-0.003406,-0.04291,0.028169,0.02093,-0.021014,-0.044312,-0.037629,…,-0.002603,0.064733,-0.029632,0.005119,0.019319,0.003828,-0.040464,0.040389,0.05743,-0.005433,-0.006662,0.012598,0.022417,-0.004811,0.044321,0.009152,0.003701,-0.076212,-0.051264,-0.04497,-0.002828,-0.026574,0.049401,0.013856,-0.003176,0.029433,-0.035817,0.008347,-0.022139,0.01779,-0.068493,0.014312,-0.003235,-0.026788,-0.029159,-0.021948,0.009429


In [12]:
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import numpy as np

# 加载预训练的模型
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 获取句子的嵌入
sentences = ["This is an example sentence.", "Each sentence is encoded."]
embeddings = model.encode(sentences)

# 查看原始嵌入的维度
print("Original embedding shape:", embeddings.shape)

# 使用 PCA 调整嵌入维度
desired_dim = 128
pca = PCA(n_components=desired_dim)
reduced_embeddings = pca.fit_transform(embeddings)

# 查看降维后的嵌入维度
print("Reduced embedding shape:", reduced_embeddings.shape)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "d:\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "d:\Python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Administrator\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Administrator\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\Ad

RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:


def ProcessNetworkErrorApply(df, geolocator, geocoder, model, vectorLength=512):
    """_summary_
    处理网络错误导致的经纬度转地址错误的pandas apply函数。
    Args:
        df (pandas.DataFrame): 输出数据集。
        geolocator (_type_): openstreetmap的geolocator。
        geocoder (_type_): baiduv3的geocoder。
        model (_type_): 地址转为向量的模型。
        vectorLength (int, optional): 第一次处理PoI数据后embedding的长度. Defaults to 512.

    Returns:
        pandas.DataFrame: _description_
    """
    if df['address'] == 'networkerror':
        print('grid:{} error.'.format(df.name))
        try:
            location = geolocator.reverse(f"{df['latitude']}, {df['longitude']}")
            df['address'] = location.address
            
            embedding = model.encode(df['address'])
            embedding = pd.Series(embedding)
            df[-vectorLength:] = embedding
            time.sleep(1)
        except:
            try:
                location = geocoder.reverse(f"{df['longitude']}, {df['latitude']}")
                df['address'] = location.raw['formatted_address']
                
                embedding = model.encode(df['address'])
                embedding = pd.Series(embedding)
                df[-vectorLength:] = embedding
                time.sleep(1)
            except:
                df['address'] = 'networkerror'
                print('grid {} both openstreetmap and baidu are network error.'.format(df.name))
        
    return df

def ProcessNetworkError(PoIFeaturePath='./Data/Output/PoIFeature.csv',
                        PoIFeatureSavePath='./Data/Output/PoIFeature.csv'):
    """_summary_
    处理网络错误导致的经纬度转地址错误。
    这个函数不在main中运行，而是等main处理完成之后，再查漏补缺的。而且每天经纬度转地址是有次数限制的。
    注意保存的路径和读入的路径是不一样的。因为不能确定在处理经纬度转地址的过程中不出现错误。
    Args:
        PoIFeaturePath (str, optional): 第一次生成PoI特征时的路径. Defaults to './Data/Output/PoIFeature.csv'.
        PoIFeatureSavePath (str, optional): 处理网络故障之后保存PoI特征时的路径. Defaults to './Data/Output/PoIFeature2.csv'.
    """
    PoIFeature = pd.read_csv(PoIFeaturePath, index_col=0)
    geolocator = Nominatim(user_agent="http")
    geocoder = BaiduV3(api_key='your ak', timeout=200)
    model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
    
    PoIFeature = PoIFeature.apply(ProcessNetworkErrorApply, geolocator=geolocator, geocoder=geocoder, 
                                  model=model, 
                                  vectorLength=512,
                                  axis=1)
    # 删除地址的文字列。
    PoIFeature.drop(columns=['address'], inplace=True)
    PoIFeature.to_csv(PoIFeatureSavePath)
    print("Completed.")

In [2]:
import pandas as pd

# "./Data/Output/usersTrajectory.csv"
df = pd.read_csv("./Data/Output/usersTrajectory.csv", nrows=10, index_col=0,
                 parse_dates=['entireTime'],
                    date_format='%Y-%m-%d %H:%M:%S')
df.head(10)

,latitude,longitude,alt,entireTime,loncol,latcol,grid,userID,0.0,1.0,...,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,longitude_right,latitude_right
46,39.903317,116.420233,101.706037,2007-10-21 10:54:51,87,56,41153,175,0.0,28.0,...,56.0,12.0,29.0,117.0,56.0,14.0,32.0,133.0,116.425126,39.903620
47,39.903550,116.419800,104.986877,2007-10-21 10:55:51,87,56,41153,175,0.0,28.0,...,56.0,12.0,29.0,117.0,56.0,14.0,32.0,133.0,116.425126,39.903620
48,39.966117,116.340800,104.986877,2007-10-21 11:42:13,80,63,41167,175,0.0,46.0,...,51.0,15.0,27.0,45.0,118.0,6.0,34.0,55.0,116.342644,39.966572
49,39.966450,116.339833,104.986877,2007-10-21 11:42:33,80,63,41167,175,0.0,46.0,...,51.0,15.0,27.0,45.0,118.0,6.0,34.0,55.0,116.342644,39.966572
50,39.966850,116.338933,104.986877,2007-10-21 11:56:06,80,63,41167,175,0.0,46.0,...,51.0,15.0,27.0,45.0,118.0,6.0,34.0,55.0,116.342644,39.966572
51,39.965567,116.338900,0.000000,2007-10-19 22:14:51,80,63,41167,175,0.0,46.0,...,51.0,15.0,27.0,45.0,118.0,6.0,34.0,55.0,116.342644,39.966572
108,39.965900,116.314717,104.986877,2007-10-22 01:47:40,78,63,40029,175,0.0,69.0,...,145.0,62.0,122.0,196.0,214.0,10.0,37.0,194.0,116.319078,39.966572
109,39.965917,116.315650,104.986877,2007-10-22 01:48:25,78,63,40029,175,0.0,69.0,...,145.0,62.0,122.0,196.0,214.0,10.0,37.0,194.0,116.319078,39.966572
110,39.966500,116.315867,104.986877,2007-10-22 01:49:25,78,63,40029,175,0.0,69.0,...,145.0,62.0,122.0,196.0,214.0,10.0,37.0,194.0,116.319078,39.966572
111,39.966917,116.314717,124.671916,2007-10-22 01:49:46,78,63,40029,175,0.0,69.0,...,145.0,62.0,122.0,196.0,214.0,10.0,37.0,194.0,116.319078,39.966572


In [3]:
df2 = pd.DataFrame()
df3 = pd.concat([df, df2])
df3

,latitude,longitude,alt,entireTime,loncol,latcol,grid,userID,0.0,1.0,...,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,longitude_right,latitude_right
46,39.903317,116.420233,101.706037,2007-10-21 10:54:51,87,56,41153,175,0.0,28.0,...,56.0,12.0,29.0,117.0,56.0,14.0,32.0,133.0,116.425126,39.903620
47,39.903550,116.419800,104.986877,2007-10-21 10:55:51,87,56,41153,175,0.0,28.0,...,56.0,12.0,29.0,117.0,56.0,14.0,32.0,133.0,116.425126,39.903620
48,39.966117,116.340800,104.986877,2007-10-21 11:42:13,80,63,41167,175,0.0,46.0,...,51.0,15.0,27.0,45.0,118.0,6.0,34.0,55.0,116.342644,39.966572
49,39.966450,116.339833,104.986877,2007-10-21 11:42:33,80,63,41167,175,0.0,46.0,...,51.0,15.0,27.0,45.0,118.0,6.0,34.0,55.0,116.342644,39.966572
50,39.966850,116.338933,104.986877,2007-10-21 11:56:06,80,63,41167,175,0.0,46.0,...,51.0,15.0,27.0,45.0,118.0,6.0,34.0,55.0,116.342644,39.966572
51,39.965567,116.338900,0.000000,2007-10-19 22:14:51,80,63,41167,175,0.0,46.0,...,51.0,15.0,27.0,45.0,118.0,6.0,34.0,55.0,116.342644,39.966572
108,39.965900,116.314717,104.986877,2007-10-22 01:47:40,78,63,40029,175,0.0,69.0,...,145.0,62.0,122.0,196.0,214.0,10.0,37.0,194.0,116.319078,39.966572
109,39.965917,116.315650,104.986877,2007-10-22 01:48:25,78,63,40029,175,0.0,69.0,...,145.0,62.0,122.0,196.0,214.0,10.0,37.0,194.0,116.319078,39.966572
110,39.966500,116.315867,104.986877,2007-10-22 01:49:25,78,63,40029,175,0.0,69.0,...,145.0,62.0,122.0,196.0,214.0,10.0,37.0,194.0,116.319078,39.966572
111,39.966917,116.314717,124.671916,2007-10-22 01:49:46,78,63,40029,175,0.0,69.0,...,145.0,62.0,122.0,196.0,214.0,10.0,37.0,194.0,116.319078,39.966572


In [4]:
pp = pd.read_csv("./Data/Output/PoIFeature.csv", nrows=5, index_col=0)
pp.head(3)

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,longitude,latitude
2541.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,1.0,115.482481,39.651810
2684.0,0.0,0.0,1.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,115.494264,39.651810
2686.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,115.482481,39.660803


In [1]:
import polars as pl
PoIFeature = pl.read_csv("./Data/Output/PoIFeature.csv", has_header=True)
PoIFeature = PoIFeature.rename({PoIFeature.collect_schema().names()[0]: 'grid'})
PoIFeature = PoIFeature.with_columns(pl.col('grid').cast(pl.Int32))
PoIFeature.columns

['grid',
 '0.0',
 '1.0',
 '2.0',
 '3.0',
 '4.0',
 '5.0',
 '6.0',
 '7.0',
 '8.0',
 '9.0',
 '10.0',
 '11.0',
 '12.0',
 '13.0',
 'longitude',
 'latitude']

In [9]:
import pandas as pd

# "./Data/Output/StayMatrix/{}.csv"
# , nrows=10

df = pd.read_csv("./Data/Output/StayMatrix/000.csv")
df

,0.0,76.0,65.0,3987.0,40033.0,43.0,3.0,297.0,4.0,10.0,...,0.0.3329,0.0.3330,0.0.3331,0.0.3332,0.0.3333,0.0.3334,0.0.3335,0.0.3336,0.0.3337,0.0.3338
0,0.0,79.0,66.0,47257.0,42327.0,45.0,0.0,308.0,4.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,78.0,68.0,71559.0,42914.0,49.0,0.0,336.0,4.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,78.0,68.0,10204.0,42914.0,14.0,0.0,89.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,78.0,68.0,22434.0,42914.0,27.0,2.0,182.0,3.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
import csv
import numpy as np
def np_3d_read_csv(path='./Data/Output/StayMatrx/{}.csv',
                   shape=(-1, 128, 3),
                   datatype='float'):
    import csv
    # 从csv文件读取2D数组
    with open(path, "r") as f:
        reader = csv.reader(f)
        a2d = np.array(list(reader)).astype(datatype)

    # 将2D数组转换为3D数组
    a = a2d.reshape(shape)
    # print(a.shape)
    return a

In [12]:
userFeature = np_3d_read_csv("./Data/Output/StayMatrix/000.csv", shape=(-1, 128, 28))

In [8]:
3584 /28

128.0

In [ ]:
def np_3d_read_csv(path='../Data/Output/StayMatrx/{}.csv',
                   shape=(-1, 128, 28),
                   datatype='float'):
    
    # 从csv文件读取2D数组
    with open(path, "r") as f:
        reader = csv.reader(f)
        a2d = np.array(list(reader)).astype(datatype)

    # 将2D数组转换为3D数组
    a = a2d.reshape(shape)
    # print(a.shape)
    return a

aa = np_3d_read_csv(path='../Data/Output/StayMatrx/132.csv')


In [4]:
def twoSum(nums: list[int], target: int) -> list[int]:
    numsLength = len(nums)
    print(numsLength)
    for i in range(numsLength):
        j = i + 1
        print(j)
        if j >= numsLength :
            return 0, 0
        else:
            for position in range(j, numsLength, 1):
                if nums[i] + nums[position] == target:
                    return i, position
                
i, j = twoSum(nums=[3,2,4], target=6)
print(i, j)

3
1
2
1 2


In [2]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)


for i in range(10):
    logging.info("This is a info message.")

2025-01-23 10:44:48,638 - INFO - This is a info message.
2025-01-23 10:44:48,640 - INFO - This is a info message.
2025-01-23 10:44:48,641 - INFO - This is a info message.
2025-01-23 10:44:48,642 - INFO - This is a info message.
2025-01-23 10:44:48,643 - INFO - This is a info message.
2025-01-23 10:44:48,645 - INFO - This is a info message.
2025-01-23 10:44:48,646 - INFO - This is a info message.
2025-01-23 10:44:48,647 - INFO - This is a info message.
2025-01-23 10:44:48,649 - INFO - This is a info message.
2025-01-23 10:44:48,650 - INFO - This is a info message.


In [4]:
import logging
import time

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

def measure_time(task_name, func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    elapsed_time = time.time() - start_time
    logging.info(f"{task_name} 执行时间: {elapsed_time:.2f} 秒")
    return result

# 示例任务
def example_task(seconds, typesrt=""):
    time.sleep(seconds)
    print(typesrt)

# 测量多个任务的执行时间
measure_time("任务 A", example_task, seconds=2, typesrt="A")
measure_time("任务 B", example_task, 3)

2025-01-23 11:23:41,988 - INFO - 任务 A 执行时间: 2.01 秒


A


2025-01-23 11:23:45,005 - INFO - 任务 B 执行时间: 3.02 秒


In [1]:
import requests

url = "https://nominatim.openstreetmap.org/reverse"
params = {"format": "json", "lat": 39.9834, "lon": 116.3229}
headers = {"User-Agent": "YourAppName"}

response = requests.get(url, params=params, headers=headers)
print(response.json())


{'place_id': 205995365, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 601749301, 'lat': '39.98285155', 'lon': '116.3245244243391', 'class': 'amenity', 'type': 'research_institute', 'place_rank': 30, 'importance': 8.344167238875436e-05, 'addresstype': 'amenity', 'name': '中国科学院物理研究所', 'display_name': '中国科学院物理研究所, 北四环西路, 科育社区, 中关村街道, 海淀区, 北京市, 100190, 中国', 'address': {'amenity': '中国科学院物理研究所', 'road': '北四环西路', 'neighbourhood': '科育社区', 'suburb': '中关村街道', 'city_district': '中关村街道', 'city': '海淀区', 'state': '北京市', 'ISO3166-2-lvl4': 'CN-BJ', 'postcode': '100190', 'country': '中国', 'country_code': 'cn'}, 'boundingbox': ['39.9812796', '39.9843887', '116.3212738', '116.3269941']}
